In [1]:
from langchain_openai import ChatOpenAI
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:

from crewai import Agent, Crew, Process, Task
from crewai_tools import SerperDevTool
from dotenv import load_dotenv
import os
load_dotenv()

# Ensure your SerperDev API key is set in the environment variables
os.environ["SERPER_API_KEY"] = os.environ["SERPER_DEV_API_KEY"]

llm_ollama= ChatOpenAI(
    model_name="llama3.2:3b",
    base_url="http://localhost:11434/v1",
)

llm_openai = ChatOpenAI(
    model_name="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
)

llm_groq = ChatOpenAI(
    model_name="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.getenv("GROQ_API_KEY"),
)

llm  = llm_openai

In [3]:
project_manager = Agent(
    role="Project Manager",
    goal="Plan the design of an electronic system for {system_requirement}",
    backstory="You're work at renesas electronics in the systems and solutions team as a project manager."
              "You have been given to plan the project timelines, manpower required and cost for designing a - "
              "{system_requirement}."
              "You can collect the information from available 'Winning combinations' on Renesas website (renesas.com)"
              "You can search the internet for latest information using the serper dev tool provdied to you."
              "You can delegate financial planning to Financial Planner Agent and Manpower planning to HR Manager Agent",
    allow_delegation=True,
    llm=llm,
    tools = [SerperDevTool()],
	verbose=True,
    max_retry_limit=3
)

In [4]:
budegting = Task(
    description=(
        "1. Create a list of different hardware components required for the project from renesas.com."
        "2. Calculate the cost of these items from different vendors."
		"3. Estimate the engineers that are needed to be hired and their exerience and skill levels."
        "4. Estimate the different tasks of the project and the engineer to be assigned for the task. "
        "5. Create a table with detailed cost estimates for hardware components. You can search websites like digikey.com and mouser.com for the prices."
        "6. Create a table with detailed cost estimate for manpower hires."
    ),
    expected_output="Two tables - 1. Budget estimate for hardware and 2. Budget estimate for manpower"
        "in markdown format, ready for publication, "
        "A final cost estimate in USD.",
    agent=project_manager,
    llm=llm,
)


In [5]:
timeline_estimation = Task(
    description=(
        "1. Define the key milestones and phases of the project from initiation to completion."
        "2. Estimate the duration for each phase based on standard industry timelines and project complexity."
        "3. Identify dependencies between different tasks to ensure logical sequencing."
        "4. Allocate time buffers for risk mitigation and unforeseen delays."
        "5. Create a Gantt chart or timeline representation for better visualization."
        "6. Ensure that the estimated timeline aligns with project goals, budget constraints, and available manpower."
    ),
    expected_output="A structured timeline with key phases, durations, dependencies, and contingencies "
        "presented in markdown format, including a textual breakdown and a visual Gantt chart if applicable.",
    agent=project_manager,
    llm=llm,
)

In [6]:
crew = Crew(
    agents=[project_manager, project_manager],
    tasks=[budegting, timeline_estimation],
    verbose=0
)

In [8]:
result = crew.kickoff(inputs={"system_requirement": "smart pen design"})

# Agent: Project Manager
## Task: 1. Create a list of different hardware components required for the project from renesas.com.2. Calculate the cost of these items from different vendors.3. Estimate the engineers that are needed to be hired and their exerience and skill levels.4. Estimate the different tasks of the project and the engineer to be assigned for the task. 5. Create a table with detailed cost estimates for hardware components. You can search websites like digikey.com and mouser.com for the prices.6. Create a table with detailed cost estimate for manpower hires.


# Agent: Project Manager
## Thought: Thought: I need to gather information on the hardware components required for the smart pen design project from Renesas' website and then search for cost estimates from vendors. After that, I will estimate the manpower required for the project.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"smart pen design hardware components site:renesas.co

In [10]:
from IPython.display import Markdown
Markdown(result.raw)

## Project Timeline for Smart Pen Design

### Key Milestones and Phases
1. **Initiation Phase** 
   - Define project scope and objectives
   - Stakeholder approval of project charter
   - Duration: 2 weeks

2. **Planning Phase**
   - Budget estimation and resource allocation
   - Define project timeline and milestones
   - Identify dependencies and risks
   - Duration: 3 weeks

3. **Design Phase**
   - Create hardware and software design specifications
   - Develop prototypes
   - Duration: 6 weeks

4. **Development Phase**
   - Hardware and firmware development
   - Implementation of software components
   - Duration: 8 weeks

5. **Testing Phase**
   - Conduct unit testing, integration testing, and system testing
   - User acceptance testing
   - Duration: 4 weeks

6. **Deployment Phase**
   - Final adjustments based on testing feedback
   - Product launch and market deployment
   - Duration: 2 weeks

7. **Closure Phase**
   - Project review and documentation
   - Stakeholder feedback
   - Duration: 1 week

### Estimated Duration for Each Phase
| Phase              | Duration (Weeks) |
|--------------------|------------------|
| Initiation         | 2                |
| Planning           | 3                |
| Design             | 6                |
| Development        | 8                |
| Testing            | 4                |
| Deployment         | 2                |
| Closure            | 1                |
| **Total Duration** | **26 weeks**     |

### Dependencies
- **Initiation** must be completed before **Planning**.
- **Planning** must be completed before **Design**.
- **Design** must be finalized before **Development**.
- **Development** must be completed before **Testing**.
- **Testing** must be completed before **Deployment**.
- **Deployment** must be completed before **Closure**.

### Time Buffers for Risk Mitigation
- Add a buffer of 2 weeks at the end of **Development**.
- Add a buffer of 1 week at the end of **Testing**.

### Gantt Chart Representation
```plaintext
| Phase         | Duration | Start Week | End Week |
|---------------|----------|------------|----------|
| Initiation    | 2 weeks  | 1          | 2        |
| Planning      | 3 weeks  | 3          | 5        |
| Design        | 6 weeks  | 6          | 11       |
| Development   | 8 weeks  | 12         | 19       |
| Testing       | 4 weeks  | 20         | 23       |
| Deployment    | 2 weeks  | 24         | 25       |
| Closure       | 1 week   | 26         | 26       |
```

### Summary
This project plan outlines a structured timeline with key phases, durations, dependencies, and contingencies for the design of a smart pen. The total estimated timeline is 26 weeks, with appropriate buffers included to address potential risks and unforeseen delays, ensuring alignment with project goals and budget constraints.
```